In [3]:
%pylab inline
import rlcompleter, readline
readline.parse_and_bind('tab: complete')
import findspark
import os
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.mllib.recommendation import ALS, Rating
import matplotlib.pyplot as plt
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType

Populating the interactive namespace from numpy and matplotlib


In [6]:
data = sc.textFile('sample.txt', 10000)
data = data.map(lambda x: x.strip().split())

In [7]:
user = data.map(lambda x:x[0]).distinct().zipWithUniqueId()
song = data.map(lambda x:x[1]).distinct().zipWithUniqueId()
# user = user.flatMap(lambda x: x.items())
# # song = song.flatMap(lambda x: x.items())

In [8]:
newData = data.map(lambda x: (x[0], (x[1], x[2]))).join(user)
newData = newData.map(lambda x: (x[1][1], x[1][0][0], x[1][0][1]))
newData = newData.map(lambda x: (x[1], (x[0], x[2]))).join(song)
newData = newData.map(lambda x: (x[1][0][0], x[1][1], int(x[1][0][1])))

In [ ]:
newData.take(10)

In [15]:
udf_user = UserDefinedFunction(lambda x: user[x], IntegerType())

udf_song = UserDefinedFunction(lambda x: song[x], IntegerType())
new_df = df.withColumn('User', udf_user(df.User))
new_df = new_df.withColumn('Song', udf_song(new_df.Song))
new_df = new_df.withColumn('ListenCnt', new_df.ListenCnt.cast('int'))

new_df = new_df.rdd

# train6, validation2,test2 = newData.randomSplit([6,2,2])
# model = ALS.train(train6, rank = 10, iterations = 5)
# predictions = model.predictAll(test2.map(lambda x: (x[0], x[1])))

# new_df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])

#newData = data.map(lambda x: Rating(int(user.lookup(x[0])), int(song.lookup(x[1])), float(x[2])))
# df = df.rdd.map(lambda row: Row(**dict(row.asDict(), user=row.User)))
# df = df.withColumn(df.select("User"))
# new = df

#new = new.withColumn('User', sc.parallelize(new.select("User").map(lambda x: user(x[1]))))


# new_df = df.withColumn('User', udf(df.User))



In [12]:
train6, validation2,test2 = data.randomSplit([6,2,2])

In [ ]:
model = ALS.trainImplicit(train6, rank = 10, iterations = 5)